## Using LangChain to get structured outputs


In [1]:
 %xmode minimal

Exception reporting mode: Minimal


In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain_anthropic import ChatAnthropic
from langchain_ollama import ChatOllama
from langchain_fireworks import ChatFireworks

from langchain.output_parsers import XMLOutputParser, PydanticOutputParser

from typing import Optional
from pydantic import BaseModel, Field
from typing_extensions import Annotated, TypedDict

In [3]:
ANTHROPIC_API_KEY = "<API KEY>"
FIREWORKS_API_KEY = "<API KEY>"

import streamlit as st

LANGSMITH_API_KEY = st.secrets["api_keys"]["LANGSMITH_API_KEY"]
ANTHROPIC_API_KEY = st.secrets["api_keys"]["ANTHROPIC_API_KEY"]
FIREWORKS_API_KEY = st.secrets["api_keys"]["FIREWORKS_API_KEY"]

Let's start by creating a LLM model to run our structured output queries. Use a temperature of 0 to improve structured output generation (but at the cost of "creativity").


In [4]:
# LLM parameters
temperature = 0.8
timeout = 30
num_ctx = 8192
num_predict = 4096

Define a LLM model below


In [5]:
# llm_model = ChatAnthropic(model="claude-3-5-haiku-20241022", api_key=claude_api_key)
llm_model = (
    ChatOllama(
        model="phi3",
        temperature=temperature,
        num_ctx=num_ctx,
        num_thread=1,
        num_predict=num_predict,
    ),
)
# llm_model = ChatFireworks(
#     # model_name="accounts/fireworks/models/llama-v3p1-70b-instruct",
#     # model_name="accounts/fireworks/models/llama-v3p2-3b-instruct",
#     model_name="accounts/fireworks/models/llama-v3p3-70b-instruct",
#     # model_name="accounts/fireworks/models/deepseek-r1",
#     api_key=FIREWORKS_API_KEY,
# )

Check it works


In [6]:
print(llm_model.invoke("Tell me a joke about zebras").content)

Why did the zebra bring a ladder to work?

Because he heard the stripes were great for climbing up the career ladder! 😄


### Problem setup and prompt


In [7]:
test_science_prompt_txt = """
You are a professional science writer tasked with responding to members of
the general public who write in asking questions about science.
Write an article responding to a writer's question for publication in a
science magazine intended for a general readership with a high-school education.
You should write clearly and compellingly, include all relavent context,
and provide motivating stories where applicable.

Your response must be less than 200 words.

The question given to you is the following:
{question}
"""

questions = [
    "What is the oldest recorded fossil?",
    "What is a black hole?",
    "How far away is the sun?",
    "Which other planet in the Solar System has a surface gravity closest to that of the Earth?",
    "Eris, Haumea, Makemake and Ceres are all examples of what?",
    "Why does earth have seasons? Do other planets exhibit seasons too?",
    "What causes the aurora borealis?",
    "Why is the sky blue?",
    "How do bees communicate?",
    "What is the smallest unit of life?",
    "How do plants make their own food?",
    "Why do we dream?",
    "What is the theory of relativity?",
    "How do volcanoes erupt?",
    "What is the speed of light?",
    "How do magnets work?",
    "What is the purpose of DNA?",
    "What are the different types of galaxies?",
    "Why do some animals hibernate?",
    "How do vaccines work?",
]

In [8]:
prompt_system_format = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Write an article in the provided format that answers the user question.\n{format_instructions}",
        ),
        ("human", test_science_prompt_txt),
    ]
)

XML output parsers


In [9]:
from langchain_core.output_parsers import XMLOutputParser

### Manual XML schema specification


In [10]:
prompt_xml_format = """
Your answers must strictly adhere to the following XML format:

<article>
<title>
[Title of the article.]
</title>

<problem>
[Summary of the writer's question, write concisely.]
</problem>

<answer>
[Answer the writer's question.]
</answer>
</article>
"""

In [11]:
xml_parser = XMLOutputParser()

In [12]:
test_chain = (
    prompt_system_format.partial(format_instructions=prompt_xml_format)
    | llm_model
    | xml_parser
)

output = test_chain.invoke(dict(question=questions[0]))

In [13]:
output

{'article': [{'title': "The Oldest Recorded Fossil: A Glimpse into Earth's Ancient Past"},
  {'problem': '\nA reader asks about the oldest known fossil, seeking to understand its significance and what it reveals about early life on Earth.\n'},
  {'answer': "\nThe oldest recorded fossil is Stromatolite fossils found in Australia, estimated to be around 3.5 billion years old. These layered structures are created by the growth of cyanobacteria, a type of photosynthetic microorganism. Discovered in the Pilbara region, these stromatolites provide crucial evidence about early life on Earth.\n\nCyanobacteria played a vital role in transforming our planet's atmosphere through oxygen production, paving the way for more complex life forms to evolve. The discovery of these ancient fossils not only offers insights into biological processes from billions of years ago but also underscores the resilience and adaptability of life. Their existence suggests that early Earth had conditions suitable for m

### LangChain Output Parsers


In [14]:
xml_parser = XMLOutputParser(
    name="article", tags=["article", "title", "problem", "answer"]
)

In [15]:
test_chain = (
    prompt_system_format.partial(
        format_instructions=xml_parser.get_format_instructions()
    )
    | llm_model
    | xml_parser
)

output = test_chain.invoke(dict(question=questions[0]))

In [16]:
output

{'article': [{'title': "The Oldest Recorded Fossil: A Glimpse into Earth's Ancient Past"},
  {'problem': '\n        Unraveling Earth’s earliest history involves studying fossils, which are preserved remains or traces of ancient organisms. The oldest recorded fossil provides a window into life billions of years ago, offering insights into the development of early life forms on our planet.\n    '},
  {'answer': "\n        The oldest known fossils are stromatolites, layered structures formed by the activities of microorganisms like cyanobacteria. These ancient formations date back approximately 3.5 billion years and have been found in rocks from Western Australia's Pilbara region. Stromatolites were crucial as they contributed to oxygenating Earth’s atmosphere through photosynthesis.\n\n        One of the most compelling discoveries is the fossilized microorganisms found in Greenland's Apex Chert, dated at about 3.7 billion years old. These tiny fossils resemble modern cyanobacteria and a

In [23]:
print(xml_parser.get_format_instructions())

The output should be formatted as a XML file.
1. Output should conform to the tags below.
2. If tags are not given, make them on your own.
3. Remember to always open and close all the tags.

As an example, for the tags ["foo", "bar", "baz"]:
1. String "<foo>
   <bar>
      <baz></baz>
   </bar>
</foo>" is a well-formatted instance of the schema.
2. String "<foo>
   <bar>
   </foo>" is a badly-formatted instance.
3. String "<foo>
   <tag>
   </tag>
</foo>" is a badly-formatted instance.

Here are the output tags:
```
['article', 'title', 'problem', 'answer']
```


### Pydantic XML would be nice!


In [17]:
from pydantic_xml import BaseXmlModel, element


class Article(BaseXmlModel, tag="article"):
    title: str = element(default="[Title of the article]")
    problem: str = element(
        default="[Summary of the writer's question, write concisely]"
    )
    answer: str = element(default="[Answer the writer's question]")

In [18]:
print(Article().to_xml(pretty_print=True).decode())

<article>
  <title>[Title of the article]</title>
  <problem>[Summary of the writer's question, write concisely]</problem>
  <answer>[Answer the writer's question]</answer>
</article>



In [19]:
xml_schema = Article().to_xml(pretty_print=True).decode()

prompt_xml_format_pyxml = f"""
Your answers must strictly adhere to the following XML format:

{xml_schema}
"""

But we can't use the `PydanticOutputParser` with a Pydantic XML object


In [24]:
pydantic_parser = PydanticOutputParser(pydantic_object=Article)

In [21]:
test_chain = (
    prompt_system_format.partial(format_instructions=prompt_xml_format_pyxml)
    | llm_model
    | pydantic_parser
)

output = test_chain.invoke(dict(question=questions[0]))

OutputParserException: Invalid json output: <article>
  <title>Unearthing Time: The Oldest Known Fossil</title>
  <problem>The writer inquires about the identity of the oldest known fossil.</problem>
  <answer>The quest for Earth's earliest life forms leads us to remarkable fossils. The oldest recorded fossil is a stromatolite, dating back approximately 3.5 billion years. Stromatolites are layered structures formed by the growth of cyanobacteria, ancient microorganisms that played a pivotal role in oxygenating our planet.

Discovered in Western Australia's Pilbara region, these fossils offer a window into early life on Earth. They suggest that life began not long after the planet cooled enough for water to exist in liquid form. These formations are significant because they provide evidence of some of the first complex life forms, shaping the course of biological evolution.

The discovery of such ancient life highlights the resilience and adaptability of organisms, sparking curiosity ab